In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
import pickle
import random
import numpy as np
import sys
import glob
import pandas as pd

In [2]:
wepick_data_header = [
"v", "u", "seq", "rgtme", "dt", "label", "av", "bq", "dn", "dot", "dv", "dvcid", "g", "lid0",
"lid1", "lid2", "s", "ci", "dgid", "ef", "ls", "pe", "po", "pot", "ps", "set", "sst", "st",
"ti1", "ti2", "ti3", "ti4", "ti5", "tn1", "tn2", "tn3", "tn4", "tn5"
]

In [3]:
data_dir = r'c:\Users\wmp\TensorFlow\DIN_tf_eager'
dic = {}
for fname in glob.glob(os.path.join(data_dir,'data*.csv')):
    df = pd.read_csv(fname, header=None, names=wepick_data_header)
    dic[fname] = df

x = pd.concat(dic.values(), ignore_index=True)

x = x[['v','u','seq', 'rgtme','dt', 'label', 'ti1', 'ti2', 'tn1', 'tn2']]

# There may be NA in ti1, ti2 (배송2.0 관련?)
x = x.reset_index(drop=True)
x = x.dropna(how='any')

x['ti1'] = x['ti1'].astype('int64')
x['ti2'] = x['ti2'].astype('int64')

In [4]:
def build_map(df, col_name):
  key = sorted(df[col_name].unique().tolist())
  m = dict(zip(key, range(len(key))))
  df[col_name] = df[col_name].map(lambda x: m[x])
  return m, key

In [5]:
origin_x = x.copy()

In [6]:
deal_to_tn1 = dict(zip(origin_x['v'], origin_x['tn1']))
deal_to_tn2 = dict(zip(origin_x['v'], origin_x['tn2']))

In [7]:
deal_map, deal_key = build_map(x, 'v')

user_map, user_key = build_map(x, 'u')

ti1_map, ti1_key = build_map(x, 'ti1')
ti2_map, ti2_key = build_map(x, 'ti2')

In [8]:
x = x.sort_values(['u','rgtme'])
x = x.reset_index(drop=True)

In [9]:
# 오류 수정: v, t1으로 groupby 한 후, 값을 취하여야 한다.
ti1_list = list(map(lambda x: x[1],[a for a, _ in x.groupby(['v', 'ti1'])]))
ti2_list = list(map(lambda x: x[1],[a for a, _ in x.groupby(['v', 'ti2'])]))

In [10]:
x = x.drop(columns=['ti1', 'ti2', 'dt', 'tn1', 'tn2'])
pos = x[x['label']==1]
neg = x[x['label']==0]

x = pd.merge(pos, neg, on=['u','rgtme'])

In [11]:
# dictionary for lowest rank given deal.
deal_slot = origin_x[['v', 'seq']]
deal_slot = deal_slot.groupby('v').min()
deal_slot = deal_slot['seq']
deal_slot = deal_slot.to_dict()

In [12]:
wepick_data = {'data':x, 
          'deal_map':deal_map, 'deal_key':deal_key,
          'user_map':user_map, 'user_key':user_key, 
          'ti1_map':ti1_map, 'ti1_key':ti1_key,
          'ti2_map':ti2_map, 'ti2_key':ti2_key,
          'ti1_list':ti1_list, 'ti2_list':ti2_list,
          'deal_slot':deal_slot
         }

with open(os.path.join(data_dir,'wepick_data.pkl'), 'wb') as f:
    pickle.dump(wepick_data, f, pickle.HIGHEST_PROTOCOL)
    f.close()

In [13]:
train_set = []
test_set = []

# restriction is needed for DIN
max_history = 24
for u, hist in x.groupby('u'):
    pos = hist['v_x'].tolist()
    neg = hist['v_y'].tolist()
    for i in range(1, len(pos)):
        hist = pos[:i]
        hist = hist[-max_history:]
        if i != len(pos)-1:
            train_set.append((u, hist, pos[i], 1))
            train_set.append((u, hist, neg[i], 0))
        else:
            label = (pos[i], neg[i])
            test_set.append((u, hist, label))

In [14]:
random.seed(1234)

random.shuffle(train_set)
random.shuffle(test_set)


In [15]:
len(train_set), len(test_set)

(924410, 240012)

In [16]:
with open(os.path.join(data_dir,'wepick_dataset.pkl'), 'wb') as f:
    pickle.dump(train_set,f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(test_set,f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(ti1_list,f, pickle.HIGHEST_PROTOCOL)
    pickle.dump((len(user_map), len(deal_map), len(ti1_map)), f, pickle.HIGHEST_PROTOCOL)


In [17]:
with open(os.path.join(data_dir,'deal_to_tn.pkl'), 'wb') as f:
    pickle.dump(deal_to_tn1,f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(deal_to_tn2,f, pickle.HIGHEST_PROTOCOL)


## logs from training

```Epoch 16 DONE	Cost time: 5413.38
Epoch 17 Global_step 492000	Train_loss: 0.4010	Eval_GAUC: 0.8195	Eval_AUC: 0.8254
Epoch 17 Global_step 493000	Train_loss: 0.4505	Eval_GAUC: 0.8212	Eval_AUC: 0.8271
Epoch 17 Global_step 494000	Train_loss: 0.4460	Eval_GAUC: 0.8204	Eval_AUC: 0.8266
Epoch 17 Global_step 495000	Train_loss: 0.4463	Eval_GAUC: 0.8206	Eval_AUC: 0.8270
Epoch 17 Global_step 496000	Train_loss: 0.4467	Eval_GAUC: 0.8210	Eval_AUC: 0.8273
Epoch 17 Global_step 497000	Train_loss: 0.4469	Eval_GAUC: 0.8189	Eval_AUC: 0.8247
Epoch 17 Global_step 498000	Train_loss: 0.4487	Eval_GAUC: 0.8219	Eval_AUC: 0.8267
Epoch 17 Global_step 499000	Train_loss: 0.4497	Eval_GAUC: 0.8199	Eval_AUC: 0.8254
Epoch 17 Global_step 500000	Train_loss: 0.4481	Eval_GAUC: 0.8252	Eval_AUC: 0.8307
Epoch 17 Global_step 501000	Train_loss: 0.4459	Eval_GAUC: 0.8220	Eval_AUC: 0.8282
Epoch 17 Global_step 502000	Train_loss: 0.4459	Eval_GAUC: 0.8233	Eval_AUC: 0.8285
Epoch 17 Global_step 503000	Train_loss: 0.4456	Eval_GAUC: 0.8232	Eval_AUC: 0.8291
Epoch 17 Global_step 504000	Train_loss: 0.4470	Eval_GAUC: 0.8216	Eval_AUC: 0.8282
Epoch 17 Global_step 505000	Train_loss: 0.4465	Eval_GAUC: 0.8194	Eval_AUC: 0.8251
Epoch 17 Global_step 506000	Train_loss: 0.4467	Eval_GAUC: 0.8216	Eval_AUC: 0.8274
Epoch 17 Global_step 507000	Train_loss: 0.4446	Eval_GAUC: 0.8191	Eval_AUC: 0.8250
Epoch 17 Global_step 508000	Train_loss: 0.4439	Eval_GAUC: 0.8185	Eval_AUC: 0.8246
Epoch 17 Global_step 509000	Train_loss: 0.4446	Eval_GAUC: 0.8180	Eval_AUC: 0.8234
Epoch 17 Global_step 510000	Train_loss: 0.4450	Eval_GAUC: 0.8208	Eval_AUC: 0.8268
Epoch 17 Global_step 511000	Train_loss: 0.4440	Eval_GAUC: 0.8204	Eval_AUC: 0.8264```